In [113]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [114]:
import tensorflow as tf
from tensorflow import keras


# Simple Regressions Made Complex
## Goal
- I will recreate Linear Regression and Logistic Regression models, however using the much more complex approach, which is through Tensorflow's **custom layers and models** and **Functional API**  

## Purpose
- The sole purpose of this is to better my understanding of the more advanced Tensorflow methods


### Use TF's **Custom Layer** to build simple densely connected layers, which are the core of Linear Regression and NNs

In [115]:
class LinearRegressionLayer(tf.keras.layers.Layer): # custom layer
    def __init__(self, units, activation=None):
        super().__init__()
        self.units = units
        self.activation = tf.keras.activations.get(activation) # activation for Logistic Regression
        
    def build(self, input_shape):  # STATE of the layer (weights, biases)
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name='weights',
                             initial_value=w_init(shape=(input_shape[-1], self.units), dtype='float32'), 
                             trainable=True)
        
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name='bias',
                             initial_value=b_init(shape=(self.units), dtype='float32'),
                             trainable=True)
        
    def call(self, inputs):   # COMPUTATION of the layer
        return self.activation( tf.matmul(inputs, self.w) + self.b )
    

### Test to see if the custom layer works

In [116]:
# load MNIST
mnist = tf.keras.datasets.mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0 # normalize

# build a simple Sequential Model
# 2 Dense layers, with 128 -> 10 units, with Dropout(0.2) in the middle
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    LinearRegressionLayer(units=128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    LinearRegressionLayer(units=10, activation='softmax')
])

# Model compilation
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Training
model.fit(X_train, y_train, epochs=3)
model.evaluate(X_test, y_test)

- Since the our simple model works with no error (though fairly poor results, but that's not the point), we can be confident that the custom layer works as intended, and can be used for the next part:

### Use this custom layer as the core of the Linear Regression and Logistic Regression models

In [117]:
class LinearRegression(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.lr = LinearRegressionLayer(units=1)
        
    def call(self, inputs):
        x = self.lr(inputs)
        return x
    
class LogisticRegression(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.logit = LinearRegressionLayer(units=1, activation='sigmoid')
    
    def call(self, inputs):
        x = self.logit(inputs)
        return x

### Test to see if our custom LR model works as intended:

In [118]:
X = np.linspace(1, 10, 10)
y = 2*X - 1 # y = 2x - 1

model = LinearRegression()
model.compile(optimizer='sgd',
              loss='mse',
              metrics=['mse'])
model.fit(X, y, epochs=300, verbose=0)
print('---------------------')
model.predict([4.])


In [119]:
model.variables # y = 1.9x - 0.6